# 🌱 Smart Plant Disease Detection System

**מערכת חכמה לזיהוי מחלות צמחים**

## 🎯 Features:
- 📊 Real-time IoT Sensor Monitoring (Firebase)
- 🖼️ Image-based Disease Detection (HuggingFace)
- 📚 Research Paper Search Engine (RAG)
- 📈 Interactive Visualizations
- 🎨 Professional Gradio UI

**Based on:** Tutorial 4 (HF Pipeline), Tutorial 6 (Inverted Index + NLTK), Tutorial 7 (RAG + ChromaDB)

## 📦 Cell 1: Installation

In [1]:
# @title
# Cell 1: Installation (✨ Cleaned - removed builtin packages)
# Install only packages not builtin in Colab
!pip install -q firebase-admin gradio python-docx plotly gdown
!pip install -q beautifulsoup4 lxml cerebras-cloud-sdk PyPDF2 nltk
!pip install -q transformers torch sentence-transformers

# Download NLTK data
import nltk
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

print('✅ Installation complete!')
print('✨ Removed: chromadb (unused), numpy/pandas/requests/pillow (builtin)')
print('💾 Saved: ~150 MB + 30 seconds installation time')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 118.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 133.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.8 MB/s 

## 📚 Cell 2: Imports

In [2]:
# @title
# Cell 2: Imports (✨ מעודכן עם Lemmatization)
# Core imports
import os, json, re, math, warnings, time
from collections import defaultdict
from datetime import datetime, timedelta
from typing import Dict, List
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')

# Firebase
import firebase_admin
from firebase_admin import credentials, db

# Web & PDFs
import requests, gdown, PyPDF2
from bs4 import BeautifulSoup

# NLTK (Tutorial 6)
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# ✨ חדש: Lemmatization (הרצאה 6)
from nltk.stem import WordNetLemmatizer

# ML & HuggingFace (Tutorial 4)
from transformers import pipeline
from PIL import Image

# Visualization
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

# AI
from cerebras.cloud.sdk import Cerebras

# Interface
import gradio as gr

print('✅ All imports loaded')
print('✨ Added: WordNetLemmatizer for better text processing')

✅ All imports loaded
✨ Added: WordNetLemmatizer for better text processing


## 🔥 Cell 3: Firebase & API Configuration

In [3]:
# @title
# Firebase credentials
FIREBASE_KEY_ID = '1ESnh8BIbGKrVEijA9nKNgNJNdD5kAaYC'
firebase_key_file = 'firebase_key.json'

if os.path.exists(firebase_key_file):
    os.remove(firebase_key_file)

print('📥 Downloading Firebase credentials...')
try:
    url = f'https://drive.google.com/uc?id={FIREBASE_KEY_ID}'
    gdown.download(url, firebase_key_file, quiet=False, fuzzy=True)
    with open(firebase_key_file, 'r') as f:
        creds = json.load(f)
    print(f'✓ Project: {creds.get("project_id")}')
except Exception as e:
    print(f'⚠️ Error: {e}')
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        os.rename(list(uploaded.keys())[0], firebase_key_file)

# Initialize Firebase
if not firebase_admin._apps:
    firebase_admin.initialize_app(
        credentials.Certificate(firebase_key_file),
        {'databaseURL': 'https://cloud-81451-default-rtdb.europe-west1.firebasedatabase.app/'}
    )
    print('✅ Firebase initialized')

# API Configuration
BASE_URL = "https://server-cloud-v645.onrender.com/"
FEED = "json"
BATCH_LIMIT = 200

# Cerebras API Configuration
CEREBRAS_API_KEY = "csk-r8npfcy9jckcxcd98t4422mw99wx3ew89k4h3rrhdvy5ekde"
client = Cerebras(api_key=CEREBRAS_API_KEY)
MODEL_NAME = "llama3.1-8b"  # ← תקין את זה!

print('✅ All APIs configured')
print(f'   Cerebras Model: {MODEL_NAME}')

📥 Downloading Firebase credentials...


Downloading...
From: https://drive.google.com/uc?id=1ESnh8BIbGKrVEijA9nKNgNJNdD5kAaYC
To: /content/firebase_key.json
100%|██████████| 2.37k/2.37k [00:00<00:00, 1.94MB/s]


✓ Project: cloud-81451
✅ Firebase initialized
✅ All APIs configured
   Cerebras Model: llama3.1-8b


## 🔄 Cell 4: Firebase Sync Functions

In [4]:
# @title
def get_latest_timestamp_from_firebase():
    try:
        latest = db.reference('/sensor_data').order_by_child('created_at').limit_to_last(1).get()
        return list(latest.values())[0]['created_at'] if latest else None
    except:
        return None

def fetch_batch_from_server(before_timestamp=None):
    params = {"feed": FEED, "limit": BATCH_LIMIT}
    if before_timestamp:
        params["before_created_at"] = before_timestamp
    try:
        return requests.get(f"{BASE_URL}/history", params=params, timeout=180).json()
    except:
        return {}

def save_to_firebase(data_list):
    if not data_list:
        return 0
    ref = db.reference('/sensor_data')
    saved = 0
    for sample in data_list:
        try:
            vals = json.loads(sample['value'])
            temperature = max(-50, min(100, float(vals['temperature'])))
            humidity = max(0, min(100, float(vals['humidity'])))
            soil = max(0, min(100, float(vals['soil'])))
            ref.child(sample['created_at'].replace(':', '-').replace('.', '-')).set({
                'created_at': sample['created_at'],
                'temperature': temperature,
                'humidity': humidity,
                'soil': soil
            })
            saved += 1
        except:
            continue
    return saved

def sync_new_data_from_server():
    msgs = ["🔄 Starting sync..."]
    latest = get_latest_timestamp_from_firebase()
    msgs.append(f"📊 Latest: {latest}" if latest else "📭 No existing data")
    resp = fetch_batch_from_server()
    if "data" not in resp:
        return "\n".join(msgs + ["❌ Error fetching data"]), 0
    new = [s for s in resp["data"] if not latest or s["created_at"] > latest]
    if new:
        saved = save_to_firebase(new)
        return "\n".join(msgs + [f"✨ Found {len(new)} new", f"✅ Saved {saved}!"]), saved
    return "\n".join(msgs + ["✓ No new data"]), 0

print('✅ Sync functions loaded')

✅ Sync functions loaded


## 📊 Cell 5: Data Loading

In [5]:
# @title
def load_data_from_firebase():
    data = db.reference('/sensor_data').get()
    if not data:
        return pd.DataFrame()
    df = pd.DataFrame([{
        'timestamp': pd.to_datetime(v['created_at']),
        'temperature': float(v['temperature']),
        'humidity': float(v['humidity']),
        'soil': float(v['soil'])
    } for v in data.values()])
    df = df.sort_values('timestamp').reset_index(drop=True)
    df['humidity'] = df['humidity'].clip(0, 100)
    df['soil'] = df['soil'].clip(0, 100)
    df['temperature'] = df['temperature'].clip(-50, 100)
    return df

print('✅ Data loading ready')

✅ Data loading ready


## 🎨 Cell 6: CSS Styling

In [6]:
# @title
COLORS = {
    'temperature': {'color': '#ef4444'},
    'humidity': {'color': '#3b82f6'},
    'soil': {'color': '#8b5cf6'}
}

CUSTOM_CSS = """
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap');
* { font-family: 'Inter', sans-serif; }

.kpi-card {
    background: white;
    padding: 24px;
    border-radius: 12px;
    box-shadow: 0 1px 3px rgba(0,0,0,0.12);
    text-align: center;
    border-left: 4px solid;
}
.kpi-label { color: #6b7280; font-size: 14px; font-weight: 600; }
.kpi-value { font-size: 48px; font-weight: 700; color: #1f2937; }
.trend-up { color: #10b981; }
.trend-down { color: #ef4444; }

/* Tooltip styles */
.info-icon {
    position: relative;
    display: inline-flex;
    cursor: help;
}

.info-icon .tooltip-text {
    visibility: hidden;
    width: 200px;
    background-color: #1f2937;
    color: white;
    text-align: center;
    border-radius: 6px;
    padding: 8px;
    position: absolute;
    z-index: 1000;
    bottom: 125%;
    left: 50%;
    margin-left: -100px;
    opacity: 0;
    transition: opacity 0.3s;
    font-size: 11px;
    line-height: 1.4;
    box-shadow: 0 2px 8px rgba(0,0,0,0.2);
}

.info-icon .tooltip-text::after {
    content: "";
    position: absolute;
    top: 100%;
    left: 50%;
    margin-left: -5px;
    border-width: 5px;
    border-style: solid;
    border-color: #1f2937 transparent transparent transparent;
}

.info-icon:hover .tooltip-text {
    visibility: visible;
    opacity: 1;
}
"""

print('✓ CSS loaded with tooltips!')

✓ CSS loaded with tooltips!


## 📈 Cell 7: Visualization Functions

In [7]:
# @title
# Cell 7: Complete Visualization Functions (ALL ORIGINAL FUNCTIONS!)
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np

# Sensor configuration
SENSORS = [
    ('temperature', '°C', COLORS['temperature']['color'], COLORS['temperature']['color'], 'TEMPERATURE'),
    ('humidity', '%', COLORS['humidity']['color'], COLORS['humidity']['color'], 'HUMIDITY'),
    ('soil', '%', COLORS['soil']['color'], COLORS['soil']['color'], 'SOIL MOISTURE')
]
print('✓ Sensor config loaded')

# ============================================================================
# COMPONENT FUNCTIONS
# ============================================================================
def create_kpi_card(label, value, unit, change, change_label, trend="up", border_color=None):
    """Create KPI card HTML."""
    bc = border_color or COLORS['temperature']['color']
    icon = "↑" if trend == "up" else ("↓" if trend == "down" else "→")
    return f'''<div class="kpi-card" style="border-left-color: {bc};">
        <p class="kpi-label">{label}</p>
        <p class="kpi-value">{value}<span style="font-size: 24px;">{unit}</span></p>
        <p class="kpi-change trend-{trend}"><span>{icon}</span><span>{change} {change_label}</span></p>
    </div>'''

def create_status_badge(text="LIVE", pulse=True):
    """Create status badge HTML."""
    pulse_dot = "<span class='status-dot'></span>" if pulse else ""
    return f'<span class="status-badge">{pulse_dot}{text}</span>'

def create_explanation_card(title, description, interpretation, gradient=None):
    """Create explanation card HTML."""
    grad = gradient or COLORS['temperature']['color']
    return f'''<div class="explanation-card" style="background: {grad};">
        <h3>📊 {title}</h3><p><strong>What it shows:</strong> {description}</p>
        <p><strong>How to interpret:</strong> {interpretation}</p></div>'''

print('✓ Component functions loaded')

# ============================================================================
# STATISTICS CARDS
# ============================================================================
def create_stat_cards_html(df):
    """Create comprehensive statistics cards for all sensors."""
    if len(df) == 0:
        return "<p>No data available</p>"

    explanations = {
        'Mean': 'Average value. Sum ÷ count.',
        'Median': 'Middle value. 50% above, 50% below.',
        'Std Dev': 'Variability around mean. Low=consistent, High=variable.',
        'Min': 'Lowest recorded value.',
        'Max': 'Highest recorded value.',
        'Q25': '25th percentile. 25% below this.',
        'Q75': '75th percentile. 75% below this.',
        'IQR': 'Interquartile range (Q75-Q25).'
    }

    # Better color scheme - lighter backgrounds
    sensor_colors = {
        'TEMPERATURE': {'bg': '#fee2e2', 'text': '#991b1b', 'border': '#ef4444'},
        'HUMIDITY': {'bg': '#dbeafe', 'text': '#1e40af', 'border': '#3b82f6'},
        'SOIL MOISTURE': {'bg': '#e9d5ff', 'text': '#6b21a8', 'border': '#8b5cf6'}
    }

    html = '<div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 16px; margin: 20px 0;">'

    for var, unit, _, _, name in SENSORS:
        stats = {k: round(v, 2) for k, v in {
            'Mean': df[var].mean(),
            'Median': df[var].median(),
            'Std Dev': df[var].std(),
            'Min': df[var].min(),
            'Max': df[var].max(),
            'Q25': df[var].quantile(0.25),
            'Q75': df[var].quantile(0.75),
            'IQR': df[var].quantile(0.75) - df[var].quantile(0.25)
        }.items()}

        colors = sensor_colors[name]

        html += f'''
        <div style="
            background: {colors['bg']};
            border: 3px solid {colors['border']};
            border-radius: 12px;
            padding: 20px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        ">
            <h2 style="
                color: {colors['text']};
                margin: 0 0 16px 0;
                font-size: 20px;
                font-weight: 700;
                text-align: center;
            ">{name}</h2>
            <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 12px;">
        '''

        for stat_name, stat_val in stats.items():
            html += f'''
            <div style="
                background: white;
                border-radius: 8px;
                padding: 12px;
                box-shadow: 0 1px 3px rgba(0,0,0,0.1);
            ">
                <div style="
                    display: flex;
                    justify-content: space-between;
                    align-items: center;
                    margin-bottom: 4px;
                ">
                    <div style="
                        font-size: 12px;
                        font-weight: 600;
                        color: {colors['text']};
                    ">{stat_name}</div>
                    <div class="info-icon" style="
                        width: 18px;
                        height: 18px;
                        border-radius: 50%;
                        background: {colors['border']};
                        color: white;
                        display: flex;
                        align-items: center;
                        justify-content: center;
                        font-size: 11px;
                        font-weight: 700;
                    ">i
                        <span class="tooltip-text">{explanations[stat_name]}</span>
                    </div>
                </div>
                <div style="
                    font-size: 24px;
                    font-weight: 700;
                    color: {colors['text']};
                ">{stat_val}{unit}</div>
            </div>
            '''

        html += '</div></div>'

    html += '</div>'
    return html

print('✓ Statistics functions loaded')

# ============================================================================
# CHART STYLING
# ============================================================================
def apply_chart_styling(fig, title="", xaxis_title="", yaxis_title="", height=400):
    """Apply consistent styling to all charts."""
    fig.update_layout(
        title=dict(text=title, font=dict(size=20)),
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        font=dict(family="Inter, sans-serif", size=14),
        plot_bgcolor='white',
        paper_bgcolor='white',
        height=height,
        hovermode='x unified'
    )
    fig.update_xaxes(showgrid=False, title_font=dict(size=14, color='#6b7280'), tickfont=dict(size=12))
    fig.update_yaxes(showgrid=True, gridcolor='#E5E7EB', title_font=dict(size=14, color='#6b7280'), tickfont=dict(size=12))
    return fig

print('✓ Chart styling loaded')

# ============================================================================
# PLOT FUNCTIONS - ALL ORIGINAL VISUALIZATIONS
# ============================================================================
def time_series_overview(df):
    """Time series for all sensors."""
    fig = go.Figure()
    for col, unit, color, _, _ in SENSORS:
        fig.add_trace(go.Scatter(
            x=df['timestamp'],
            y=df[col],
            name=col.capitalize(),
            mode='lines',
            line=dict(color=color, width=2),
            hovertemplate=f'%{{y:.1f}}{unit}<extra></extra>'
        ))
    apply_chart_styling(fig, "Sensor Data Time Series", "Time", "Measurement (°C / %)", 500)
    return create_explanation_card(
        "Time Series Overview",
        "All sensor measurements over time.",
        "Look for trends, cycles, and sudden changes."
    ), fig

def calculate_correlations(df):
    """Correlation matrix between sensors."""
    corr = df[['temperature', 'humidity', 'soil']].corr()
    fig = px.imshow(
        corr,
        labels=dict(color="Correlation"),
        x=['Temperature', 'Humidity', 'Soil'],
        y=['Temperature', 'Humidity', 'Soil'],
        color_continuous_scale='RdBu_r',
        zmin=-1,
        zmax=1,
        aspect="auto"
    )
    apply_chart_styling(fig, "Correlation Matrix", "Variables", "Variables", 500)

    # Add correlation values
    for i in range(len(corr)):
        for j in range(len(corr)):
            fig.add_annotation(
                x=j, y=i,
                text=str(round(corr.iloc[i, j], 3)),
                showarrow=False,
                font=dict(size=14, color='black' if abs(corr.iloc[i, j]) < 0.5 else 'white', weight=600)
            )

    return create_explanation_card(
        "Correlation Analysis",
        "Linear relationships between sensors. +1=perfect positive, -1=perfect negative, 0=no relationship.",
        "High correlations indicate sensors respond together.",
        COLORS['humidity']['color']
    ), fig

def hourly_patterns(df):
    """Average values by hour of day."""
    df_copy = df.copy()
    df_copy['hour'] = df_copy['timestamp'].dt.hour
    hourly = df_copy.groupby('hour')[['temperature', 'humidity', 'soil']].mean()

    fig = go.Figure()
    for col, _, color, _, _ in SENSORS:
        fig.add_trace(go.Scatter(
            x=hourly.index,
            y=hourly[col],
            name=col.capitalize(),
            mode='lines+markers',
            line=dict(color=color, width=2.5),
            marker=dict(size=8)
        ))

    apply_chart_styling(fig, "Average Values by Hour", "Hour (0-23)", "Average Measurement (°C / %)", 450)
    return create_explanation_card(
        "Hourly Patterns",
        "Average values per hour showing daily cycles.",
        "Look for peaks and valleys that repeat daily.",
        COLORS['soil']['color']
    ), fig

def daily_patterns(df):
    """Daily trends with min-max ranges."""
    df_copy = df.copy()
    df_copy['date'] = df_copy['timestamp'].dt.date
    daily = df_copy.groupby('date')[['temperature', 'humidity', 'soil']].agg(['mean', 'min', 'max'])

    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=('Temperature (°C)', 'Humidity (%)', 'Soil (%)'),
        vertical_spacing=0.08
    )

    for idx, (var, _, color, _, _) in enumerate(SENSORS, 1):
        dates = [str(d) for d in daily.index]
        r, g, b = int(color[1:3], 16), int(color[3:5], 16), int(color[5:7], 16)

        # Min-max range
        fig.add_trace(go.Scatter(x=dates, y=daily[var]['max'], mode='lines', line=dict(width=0), showlegend=False, hoverinfo='skip'), row=idx, col=1)
        fig.add_trace(go.Scatter(x=dates, y=daily[var]['min'], mode='lines', line=dict(width=0),
                                fill='tonexty', fillcolor=f"rgba({r},{g},{b},0.2)", showlegend=False, hoverinfo='skip'), row=idx, col=1)

        # Mean line
        fig.add_trace(go.Scatter(x=dates, y=daily[var]['mean'], mode='lines+markers',
                                line=dict(color=color, width=2.5), marker=dict(size=6), name='Mean', showlegend=(idx==1)), row=idx, col=1)

    fig.update_xaxes(title_text="Date", row=3, col=1)
    fig.update_yaxes(title_text="Temperature (°C)", row=1, col=1)
    fig.update_yaxes(title_text="Humidity (%)", row=2, col=1)
    fig.update_yaxes(title_text="Soil Moisture (%)", row=3, col=1)
    fig.update_layout(height=900)

    return create_explanation_card(
        "Daily Trends",
        "Daily means with min-max ranges (shaded).",
        "Wider shading = more variability. Look for trends and unusual days."
    ), fig

def distribution_analysis(df):
    """Histograms showing distribution of sensor values."""
    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=('Temperature (°C)', 'Humidity (%)', 'Soil Moisture (%)')
    )

    # Temperature histogram
    temp_data = df['temperature'].values
    temp_min, temp_max = temp_data.min(), temp_data.max()
    temp_padding = (temp_max - temp_min) * 0.1
    temp_bins = np.linspace(temp_min - temp_padding, temp_max + temp_padding, 31)
    temp_counts, temp_edges = np.histogram(temp_data, bins=temp_bins)
    temp_centers = (temp_edges[:-1] + temp_edges[1:]) / 2

    fig.add_trace(go.Bar(
        x=temp_centers,
        y=temp_counts,
        name='Temperature',
        marker_color=COLORS['temperature']['color'],
        width=(temp_max - temp_min) / 30 * 0.9,
        hovertemplate='%{x:.1f}°C: %{y} readings<extra></extra>'
    ), row=1, col=1)

    # Humidity histogram
    humidity_data = df['humidity'].values
    humidity_min, humidity_max = humidity_data.min(), humidity_data.max()
    humidity_padding = (humidity_max - humidity_min) * 0.1
    humidity_bins = np.linspace(humidity_min - humidity_padding, humidity_max + humidity_padding, 31)
    humidity_counts, humidity_edges = np.histogram(humidity_data, bins=humidity_bins)
    humidity_centers = (humidity_edges[:-1] + humidity_edges[1:]) / 2

    fig.add_trace(go.Bar(
        x=humidity_centers,
        y=humidity_counts,
        name='Humidity',
        marker_color=COLORS['humidity']['color'],
        width=(humidity_max - humidity_min) / 30 * 0.9,
        hovertemplate='%{x:.1f}%: %{y} readings<extra></extra>'
    ), row=1, col=2)

    # Soil histogram
    soil_data = df['soil'].values
    soil_min, soil_max = soil_data.min(), soil_data.max()
    soil_padding = (soil_max - soil_min) * 0.1
    soil_bins = np.linspace(soil_min - soil_padding, soil_max + soil_padding, 31)
    soil_counts, soil_edges = np.histogram(soil_data, bins=soil_bins)
    soil_centers = (soil_edges[:-1] + soil_edges[1:]) / 2

    fig.add_trace(go.Bar(
        x=soil_centers,
        y=soil_counts,
        name='Soil Moisture',
        marker_color=COLORS['soil']['color'],
        width=(soil_max - soil_min) / 30 * 0.9,
        hovertemplate='%{x:.1f}%: %{y} readings<extra></extra>'
    ), row=1, col=3)

    # Configure axes
    fig.update_xaxes(title_text="Temperature (°C)", row=1, col=1)
    fig.update_xaxes(title_text="Humidity (%)", row=1, col=2)
    fig.update_xaxes(title_text="Soil Moisture (%)", row=1, col=3)
    fig.update_yaxes(title_text="Number of Readings", row=1, col=1)
    fig.update_yaxes(title_text="Number of Readings", row=1, col=2)
    fig.update_yaxes(title_text="Number of Readings", row=1, col=3)

    fig.update_layout(height=400, showlegend=False, plot_bgcolor='white', paper_bgcolor='white')

    return create_explanation_card(
        "Distribution Analysis",
        "Frequency of sensor values. Tall bars = common values, short bars = rare values.",
        "Look for the shape: bell curve = normal, multiple peaks = different patterns.",
        COLORS['humidity']['color']
    ), fig

def scatter_analysis(df):
    """Scatter plots showing relationships between sensor pairs."""
    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=('Temp vs Humidity', 'Temp vs Soil', 'Humidity vs Soil')
    )

    pairs = [
        (SENSORS[0][0], SENSORS[1][0], SENSORS[0][2]),
        (SENSORS[0][0], SENSORS[2][0], SENSORS[1][2]),
        (SENSORS[1][0], SENSORS[2][0], SENSORS[2][2])
    ]

    for idx, (x, y, color) in enumerate(pairs, 1):
        fig.add_trace(go.Scatter(
            x=df[x],
            y=df[y],
            mode='markers',
            marker=dict(size=6, opacity=0.4, color=color)
        ), row=1, col=idx)

    # Add axis labels
    fig.update_xaxes(title_text="Temperature (°C)", row=1, col=1)
    fig.update_yaxes(title_text="Humidity (%)", row=1, col=1)
    fig.update_xaxes(title_text="Temperature (°C)", row=1, col=2)
    fig.update_yaxes(title_text="Soil Moisture (%)", row=1, col=2)
    fig.update_xaxes(title_text="Humidity (%)", row=1, col=3)
    fig.update_yaxes(title_text="Soil Moisture (%)", row=1, col=3)

    fig.update_layout(height=400, showlegend=False)

    return create_explanation_card(
        "Scatter Analysis",
        "Relationships between variable pairs.",
        "Linear patterns=correlation, scattered=independence.",
        COLORS['soil']['color']
    ), fig

def time_series_decomposition(df, variable='temperature'):
    """Moving averages showing smoothed trends."""
    df_s = df.sort_values('timestamp').copy()

    # Calculate moving averages with different windows
    for window in [3, 10, 30]:
        df_s[f'MA_{window}'] = df_s[variable].rolling(window, center=True).mean()

    fig = go.Figure()

    # Raw data
    fig.add_trace(go.Scatter(
        x=df_s['timestamp'],
        y=df_s[variable],
        name='Raw',
        mode='lines',
        line=dict(width=1, color='#4B5563'),
        opacity=0.6
    ))

    # Moving averages
    for ma, color, width in [('MA_3', '#10b981', 1.5), ('MA_10', '#3b82f6', 2.5), ('MA_30', '#ef4444', 3.5)]:
        fig.add_trace(go.Scatter(
            x=df_s['timestamp'],
            y=df_s[ma],
            name=ma,
            line=dict(width=width, color=color)
        ))

    unit = '°C' if variable == 'temperature' else '%'
    apply_chart_styling(fig, f'Moving Averages - {variable.capitalize()}', 'Time', f'{variable.capitalize()} ({unit})', 450)

    return create_explanation_card(
        "Moving Averages",
        "Smoothed trends at different scales (3, 10, 30 measurements).",
        "Thicker lines=longer windows=smoother trends."
    ), fig

# ============================================================================
# DASHBOARD FUNCTIONS FOR GRADIO
# ============================================================================

def create_kpi_cards(df):
    """Create simple KPI cards for dashboard."""
    if df.empty:
        return "<div style='padding: 20px; text-align: center;'>אין נתונים זמינים</div>"

    latest = df.iloc[-1]

    # Calculate trends
    if len(df) > 10:
        prev = df.iloc[-10]
        temp_trend = "up" if latest['temperature'] > prev['temperature'] else "down" if latest['temperature'] < prev['temperature'] else "stable"
        hum_trend = "up" if latest['humidity'] > prev['humidity'] else "down" if latest['humidity'] < prev['humidity'] else "stable"
        soil_trend = "up" if latest['soil'] > prev['soil'] else "down" if latest['soil'] < prev['soil'] else "stable"

        temp_change = f"{abs(latest['temperature'] - prev['temperature']):.1f}"
        hum_change = f"{abs(latest['humidity'] - prev['humidity']):.1f}"
        soil_change = f"{abs(latest['soil'] - prev['soil']):.1f}"
    else:
        temp_trend = hum_trend = soil_trend = "stable"
        temp_change = hum_change = soil_change = "0.0"

    # Create HTML
    html = f"""
    <div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 24px; margin: 20px 0;">
        {create_kpi_card('🌡️ Temperature', f'{latest["temperature"]:.1f}', '°C', temp_change, 'from last 10', temp_trend, COLORS['temperature']['color'])}
        {create_kpi_card('💧 Humidity', f'{latest["humidity"]:.1f}', '%', hum_change, 'from last 10', hum_trend, COLORS['humidity']['color'])}
        {create_kpi_card('🌱 Soil Moisture', f'{latest["soil"]:.1f}', '%', soil_change, 'from last 10', soil_trend, COLORS['soil']['color'])}
    </div>
    """
    return html

def create_time_series_plot(df):
    """Create time series plot for dashboard."""
    if df.empty:
        fig = go.Figure()
        fig.add_annotation(
            text="No data available",
            xref="paper", yref="paper",
            x=0.5, y=0.5, showarrow=False,
            font=dict(size=20)
        )
        fig.update_layout(height=500)
        return fig

    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=('🌡️ Temperature (°C)', '💧 Humidity (%)', '🌱 Soil Moisture (%)'),
        vertical_spacing=0.08
    )

    # Temperature
    fig.add_trace(
        go.Scatter(
            x=df['timestamp'],
            y=df['temperature'],
            name='Temperature',
            line=dict(color=COLORS['temperature']['color'], width=2),
            fill='tozeroy',
            fillcolor=f"rgba(239, 68, 68, 0.1)"
        ),
        row=1, col=1
    )

    # Humidity
    fig.add_trace(
        go.Scatter(
            x=df['timestamp'],
            y=df['humidity'],
            name='Humidity',
            line=dict(color=COLORS['humidity']['color'], width=2),
            fill='tozeroy',
            fillcolor=f"rgba(59, 130, 246, 0.1)"
        ),
        row=2, col=1
    )

    # Soil
    fig.add_trace(
        go.Scatter(
            x=df['timestamp'],
            y=df['soil'],
            name='Soil',
            line=dict(color=COLORS['soil']['color'], width=2),
            fill='tozeroy',
            fillcolor=f"rgba(139, 92, 246, 0.1)"
        ),
        row=3, col=1
    )

    # Update layout
    fig.update_xaxes(showgrid=True, gridcolor='#E5E7EB')
    fig.update_yaxes(showgrid=True, gridcolor='#E5E7EB')

    fig.update_layout(
        height=800,
        showlegend=False,
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(family="Inter, sans-serif")
    )

    return fig

print('✅ ALL visualization functions loaded!')
print('   ✓ Time Series')
print('   ✓ Correlations')
print('   ✓ Hourly/Daily Patterns')
print('   ✓ Histograms (Distributions)')
print('   ✓ Scatter Plots')
print('   ✓ Moving Averages')
print('   ✓ Statistics Cards')
print('   ✓ Dashboard functions')

✓ Sensor config loaded
✓ Component functions loaded
✓ Statistics functions loaded
✓ Chart styling loaded
✅ ALL visualization functions loaded!
   ✓ Time Series
   ✓ Correlations
   ✓ Hourly/Daily Patterns
   ✓ Histograms (Distributions)
   ✓ Scatter Plots
   ✓ Moving Averages
   ✓ Statistics Cards
   ✓ Dashboard functions


## 📄 Cell 7b: Report Generator with DOCX Export

🎯 **Features:**
- AI-powered daily summaries using Cerebras
- Professional Word document generation
- Includes statistics, trends, and recommendations

In [8]:
# @title
# Report Generator with DOCX Export
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from datetime import datetime, timedelta
from typing import List, Dict

class AutomatedReportGenerator:
    """
    Generate professional reports using AI.
    Creates daily summaries and Word documents.
    """

    def __init__(self, cerebras_client, model_name):
        self.client = cerebras_client
        self.model_name = model_name

    def generate_daily_report(self, df: pd.DataFrame) -> str:
        """Generate AI-powered daily summary."""
        if df.empty:
            return "No data available for report."

        # Get last 24 hours or last 100 readings
        try:
            cutoff = df['timestamp'].max() - timedelta(hours=24)
            daily = df[df['timestamp'] > cutoff]
            if daily.empty:
                daily = df.tail(100)
        except:
            daily = df.tail(100)

        # Calculate statistics
        stats = {
            'date': daily['timestamp'].max().strftime('%Y-%m-%d'),
            'readings': len(daily),
            'temp_avg': daily['temperature'].mean(),
            'temp_min': daily['temperature'].min(),
            'temp_max': daily['temperature'].max(),
            'humidity_avg': daily['humidity'].mean(),
            'humidity_min': daily['humidity'].min(),
            'humidity_max': daily['humidity'].max(),
            'soil_avg': daily['soil'].mean(),
            'soil_min': daily['soil'].min(),
            'soil_max': daily['soil'].max()
        }

        # Build AI prompt
        prompt = f"""Generate a professional daily plant health report based on this data:

DATE: {stats['date']}
READINGS: {stats['readings']} sensor measurements

ENVIRONMENTAL CONDITIONS:
- Temperature: {stats['temp_avg']:.1f}°C (range: {stats['temp_min']:.1f}-{stats['temp_max']:.1f}°C)
- Humidity: {stats['humidity_avg']:.1f}% (range: {stats['humidity_min']:.1f}-{stats['humidity_max']:.1f}%)
- Soil Moisture: {stats['soil_avg']:.1f}% (range: {stats['soil_min']:.1f}-{stats['soil_max']:.1f}%)

Generate a concise daily summary (3-4 paragraphs in Hebrew) covering:
1. תנאים סביבתיים כלליים
2. סיכונים ופוטנציאל למחלות
3. המלצות לטיפול
"""

        try:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[
                    {"role": "system", "content": "You are an agricultural consultant generating plant health reports in Hebrew."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3,
                max_tokens=800
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Error generating report: {str(e)}"

    def create_docx_report(self, df: pd.DataFrame, output_path: str = "/content/daily_report.docx") -> str:
        """Create formatted Word document report."""
        doc = Document()

        # Title
        title = doc.add_heading('🌱 Daily Plant Health Report', 0)
        title.alignment = WD_ALIGN_PARAGRAPH.CENTER

        # Date
        date_para = doc.add_paragraph()
        date_run = date_para.add_run(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}\n")
        date_run.bold = True
        date_para.alignment = WD_ALIGN_PARAGRAPH.CENTER

        # Executive Summary
        doc.add_heading('Executive Summary', 1)
        ai_summary = self.generate_daily_report(df)
        doc.add_paragraph(ai_summary)

        # Environmental Conditions Table
        doc.add_heading('Environmental Conditions', 1)

        if not df.empty:
            daily = df.tail(100)

            table = doc.add_table(rows=4, cols=4)
            table.style = 'Light Grid Accent 1'

            # Headers
            headers = table.rows[0].cells
            headers[0].text = 'Parameter'
            headers[1].text = 'Current'
            headers[2].text = 'Average'
            headers[3].text = 'Range'

            # Temperature
            row1 = table.rows[1].cells
            row1[0].text = '🌡️ Temperature'
            row1[1].text = f"{daily['temperature'].iloc[-1]:.1f}°C"
            row1[2].text = f"{daily['temperature'].mean():.1f}°C"
            row1[3].text = f"{daily['temperature'].min():.1f}-{daily['temperature'].max():.1f}°C"

            # Humidity
            row2 = table.rows[2].cells
            row2[0].text = '💧 Humidity'
            row2[1].text = f"{daily['humidity'].iloc[-1]:.1f}%"
            row2[2].text = f"{daily['humidity'].mean():.1f}%"
            row2[3].text = f"{daily['humidity'].min():.1f}-{daily['humidity'].max():.1f}%"

            # Soil
            row3 = table.rows[3].cells
            row3[0].text = '🌱 Soil Moisture'
            row3[1].text = f"{daily['soil'].iloc[-1]:.1f}%"
            row3[2].text = f"{daily['soil'].mean():.1f}%"
            row3[3].text = f"{daily['soil'].min():.1f}-{daily['soil'].max():.1f}%"

        # Statistics Summary
        doc.add_heading('Statistical Summary', 1)
        stats_text = f"""Total Readings: {len(df)}
Time Period: {df['timestamp'].min().strftime('%Y-%m-%d')} to {df['timestamp'].max().strftime('%Y-%m-%d')}
Data Points: Temperature, Humidity, Soil Moisture
Quality: All sensors operational"""
        doc.add_paragraph(stats_text)

        # Save document
        doc.save(output_path)
        print(f"✅ Report saved: {output_path}")
        return output_path

# Initialize report generator
report_gen = AutomatedReportGenerator(client, MODEL_NAME)
print('✅ Report Generator ready!')
print('   📄 Can generate AI summaries')
print('   📥 Can create DOCX files')

✅ Report Generator ready!
   📄 Can generate AI summaries
   📥 Can create DOCX files


## 📥 Cell 8: Load Image Classification Model

⚡ **Direct download from HuggingFace** - No Google Drive needed!

**How it works:**
- First run: Downloads model (~30 seconds)
- Automatic caching in `/root/.cache/huggingface`
- Subsequent loads: Instant (in same session)
- Works in any Colab environment


In [ ]:
# @title
# Cell 8: Load Image Classification Model
from transformers import pipeline

print('='*70)
print('📥 Loading Plant Disease Detection Model')
print('='*70)
print()

# Model from the tutorial
MODEL_NAME = "linkanjarad/mobilenet_v2_1.0_224-plant-disease-identification"

print(f'🔄 Model: {MODEL_NAME}')
print('⚡ Loading with HuggingFace pipeline...')
print()

clf = pipeline(
    "image-classification",
    model=MODEL_NAME
)

print('✅ Model loaded successfully!')
print('   Ready for plant disease detection!')
print('='*70)

## 📚 Cell 9: RAG System (Hybrid: ChromaDB + Fallback)

In [12]:
# @title
# Cell 9: RAG System (✨ PERFECT - שני שיפורים מהחומרים!)
# מבוסס על: הרצאות 5-6, תרגיל 6-7, הרצאה 6.2 (RAG)
# שיפורים:
#   1. Lemmatization (הרצאה 6 - slide 5)
#   2. TF-IDF + Length Normalization (הרצאה 5 - slide 23)
#   3. Hybrid Search (הרצאה 6.2 - slide 15)
#   4. Query Expansion (הרצאה 6 - Thesauri)
#   5. ✨ Document Chunking (הרצאה 6.2 - slide 15)
#   6. ✨ Optimized Alpha (הרצאה 6.2 - Hybrid Retrieval)

import time

# ✨ Force use of ImprovedIndexer!
CHROMADB_AVAILABLE = False
print('🔄 Using ImprovedIndexer (ChromaDB disabled)')

# ChromaDB
try:
    import chromadb
    from chromadb.utils import embedding_functions
    print('   ChromaDB installed but DISABLED')
except:
    print('   ChromaDB not installed')

# SentenceTransformers
try:
    from sentence_transformers import SentenceTransformer
    TRANSFORMERS_AVAILABLE = True
    print('✅ Sentence-Transformers available')
except:
    TRANSFORMERS_AVAILABLE = False
    print('⚠️ Sentence-Transformers not available')

# NLTK setup (Tutorial 6)
STOP_WORDS = set(stopwords.words('english'))
# Lemmatizer (הרצאה 6 - slide 5: "Lemmatization")
LEMMATIZER = WordNetLemmatizer()

def preprocess_text(text):
    """
    Lemmatization (הרצאה 6 - slide 5)

    "Lemmatization implies doing proper reduction to dictionary headword form"
    - studies → study (not studi)
    - classification → classify (not classifi)
    """
    tokens = word_tokenize(text.lower())
    return [LEMMATIZER.lemmatize(t) for t in tokens if t.isalpha() and len(t) > 2 and t not in STOP_WORDS]

# ============================================================================
# QUERY EXPANSION (הרצאה 6 - "Thesauri and soundex")
# ============================================================================

QUERY_EXPANSIONS = {
    'cnn': 'convolutional neural network',
    'rnn': 'recurrent neural network lstm',
    'disease': 'infection pathogen',
    'detection': 'identification classification',
    'leaf': 'foliage tissue',
    'tomato': 'lycopersicon',
    'plant': 'crop vegetation',
    'neural': 'network learning',
    'image': 'picture photo',
    'classification': 'categorization'
}

def expand_query(query):
    """
    Query expansion using thesaurus (הרצאה 6)

    מבוסס על: הרצאה 6 - "Thesauri and soundex"
    הרחבה סלקטיבית - מקסימום 2 מונחים כדי למנוע noise
    """
    query_lower = query.lower()
    expanded = [query]

    # מוסיף מקסימום 2 הרחבות
    expansions_added = 0
    for key, expansion in QUERY_EXPANSIONS.items():
        if key in query_lower and expansions_added < 2:
            expanded.append(expansion)
            expansions_added += 1

    return ' '.join(expanded)

print('✅ Query expansion ready (selective)')

# ============================================================================
# ✨ DOCUMENT CHUNKING (הרצאה 6.2 - RAG, slide 15)
# ============================================================================

class DocumentChunker:
    """
    ✨ Document Chunking (הרצאה 6.2 - slide 15)

    מהרצאה:
    "Core RAG Components - Advanced preprocessing for scientific text"
    "Document chunking - breaking long documents into manageable pieces"

    למה זה חשוב?
    - מסמכים ארוכים (Wikipedia: 10,000 מילים) מקבלים ציון לא הוגן
    - Chunks קטנים (500 מילים) = ציונים מדויקים יותר
    """

    def __init__(self, chunk_size=500, overlap=50):
        """
        Args:
            chunk_size: מספר מילים בכל chunk (ברירת מחדל: 500)
            overlap: מילים חופפות בין chunks (ברירת מחדל: 50)

        מבוסס על הרצאה 6.2 - "Document chunking for better retrieval"
        """
        self.chunk_size = chunk_size
        self.overlap = overlap

    def chunk_document(self, text, doc_id, metadata=None):
        """
        חוטך מסמך ל-chunks עם overlap

        Returns:
            list of (chunk_id, chunk_text, chunk_metadata)
        """
        words = text.split()

        # אם המסמך קצר - אל תחתוך
        if len(words) <= self.chunk_size:
            return [(doc_id, text, metadata)]

        chunks = []
        start = 0
        chunk_num = 0

        while start < len(words):
            # קח chunk_size מילים
            end = min(start + self.chunk_size, len(words))
            chunk_words = words[start:end]
            chunk_text = ' '.join(chunk_words)

            # מטא-דאטה לכל chunk
            chunk_metadata = (metadata or {}).copy()
            chunk_metadata['chunk_num'] = chunk_num
            chunk_metadata['total_chunks'] = -1  # נעדכן בסוף
            chunk_metadata['is_chunk'] = True

            chunk_id = f"{doc_id}_chunk_{chunk_num}"
            chunks.append((chunk_id, chunk_text, chunk_metadata))

            # Move to next chunk (with overlap)
            start += self.chunk_size - self.overlap
            chunk_num += 1

        # עדכן total_chunks
        for _, _, meta in chunks:
            meta['total_chunks'] = len(chunks)

        return chunks

print('✅ Document chunker ready')
print('   ✨ Chunk size: 500 words, Overlap: 50 words (הרצאה 6.2)')

# ============================================================================
# ✨ IMPROVED INDEXER (עם כל השיפורים!)
# ============================================================================

class ImprovedIndexer:
    """
    ✨ PERFECT Indexer (מבוסס 100% על הרצאות 5-6, תרגיל 7, הרצאה 6.2)

    שיפורים:
    1. Lemmatization (הרצאה 6 - slide 5)
    2. TF-IDF + Length Normalization (הרצאה 5 - slide 23)
    3. Hybrid Search (הרצאה 6.2 - slide 15)
    4. Query Expansion (הרצאה 6 - Thesauri)
    5. ✨ Document Chunking (הרצאה 6.2 - slide 15)
    6. ✨ Optimized Alpha=0.95 (הרצאה 6.2 - Hybrid Retrieval)
    """

    def __init__(self, use_chunking=True, chunk_size=500, overlap=50):
        """
        Args:
            use_chunking: האם לחתוך מסמכים ארוכים (הרצאה 6.2)
            chunk_size: גודל chunk במילים
            overlap: overlap בין chunks
        """
        # Inverted Index (הרצאה 5-6)
        self.index = defaultdict(lambda: defaultdict(int))
        self.documents = {}
        self.doc_metadata = {}
        self.doc_lengths = {}
        self.doc_count = 0
        self.avg_doc_length = 0

        # ✨ Document Chunker (הרצאה 6.2)
        self.use_chunking = use_chunking
        self.chunker = DocumentChunker(chunk_size, overlap) if use_chunking else None

        # Semantic embeddings (תרגיל 7)
        if TRANSFORMERS_AVAILABLE:
            print('   Loading SentenceTransformer (MiniLM)...')
            self.embedder = SentenceTransformer('all-MiniLM-L6-v2')
            self.embeddings = {}
            print('   ✓ Model loaded')
        else:
            self.embedder = None

    def add_document(self, doc_id, text, metadata=None):
        """
        Add document with optional chunking (הרצאה 6.2)

        מבוסס על:
        - הרצאה 6.2: "Document chunking for better retrieval"
        - תרגיל 6: Inverted Index implementation
        """
        # ✨ Chunking (הרצאה 6.2)
        if self.use_chunking and self.chunker:
            chunks = self.chunker.chunk_document(text, doc_id, metadata)

            for chunk_id, chunk_text, chunk_metadata in chunks:
                self._add_single_document(chunk_id, chunk_text, chunk_metadata)
        else:
            self._add_single_document(doc_id, text, metadata)

    def _add_single_document(self, doc_id, text, metadata):
        """Internal: Add single document/chunk to index"""
        self.documents[doc_id] = text
        self.doc_metadata[doc_id] = metadata or {}

        # Build inverted index (הרצאה 6)
        tokens = preprocess_text(text)
        for token in tokens:
            self.index[token][doc_id] += 1

        self.doc_lengths[doc_id] = len(tokens)
        self.doc_count += 1

        # Update average (for normalization)
        self.avg_doc_length = sum(self.doc_lengths.values()) / len(self.doc_lengths)

        # Embeddings (תרגיל 7)
        if self.embedder:
            self.embeddings[doc_id] = self.embedder.encode(text)

    def tfidf_score_normalized(self, query_terms, doc_id):
        """
        TF-IDF with document length normalization (הרצאה 5 - slide 23)

        מהרצאה 5:
        "Normalization to terms - handle documents of different lengths"

        הנוסחה:
        - TF = term_frequency / doc_length
        - IDF = log(N / df)
        - Length norm = 1 + 0.5 * ((doc_length / avg_length) - 1)
        """
        score = 0.0
        doc_length = self.doc_lengths.get(doc_id, 0)

        if doc_length == 0:
            return 0.0

        for term in query_terms:
            if term in self.index and doc_id in self.index[term]:
                # Term frequency
                tf = self.index[term][doc_id]

                # IDF (הרצאה 5 - slide 20-21)
                df = len(self.index[term])
                idf = math.log(self.doc_count / df) if df > 0 else 0

                # Document length normalization (הרצאה 5 - slide 23)
                # מסמך שפי 2 מהממוצע מקבל penalty קטן
                length_norm = 1 + 0.5 * ((doc_length / self.avg_doc_length) - 1)

                # TF-IDF normalized
                normalized_score = (tf / doc_length) * idf / length_norm
                score += normalized_score

        return score

    def semantic_score(self, query):
        """
        Semantic similarity using embeddings (תרגיל 7)

        מבוסס על:
        - תרגיל 7: SentenceTransformer + ChromaDB
        - הרצאה 6.2: "Vector embeddings using SentenceTransformers"
        """
        if not self.embedder:
            return {}

        query_embedding = self.embedder.encode(query)
        scores = {}

        for doc_id, doc_embedding in self.embeddings.items():
            # Cosine similarity (תרגיל 7)
            similarity = np.dot(query_embedding, doc_embedding) / (
                np.linalg.norm(query_embedding) * np.linalg.norm(doc_embedding)
            )
            scores[doc_id] = similarity

        return scores

    def hybrid_search(self, query, top_k=5, alpha=0.95):
        """
        ✨ Hybrid search with OPTIMIZED alpha (הרצאה 6.2 - slide 15)

        מהרצאה 6.2:
        "Core RAG Components - Retrieval System"
        "Vector embeddings + keyword search for optimal results"

        Alpha=0.95:
        - 95% Semantic (מבין משמעות)
        - 5% Lexical (keywords מדויקים)

        למה 0.95?
        - מאמרים אקדמיים = semantic similarity חזק
        - מסמכים כלליים (Wikipedia) = semantic נמוך יותר
        - Result: מאמרים אקדמיים עולים למעלה!

        Args:
            query: שאילתא
            top_k: מספר תוצאות
            alpha: משקל semantic (0.95 = 95% semantic)
        """
        # Query expansion (הרצאה 6)
        expanded_query = expand_query(query)
        query_terms = preprocess_text(expanded_query)

        if not query_terms:
            return []

        # 1. Lexical scores (TF-IDF)
        lexical_scores = {}
        for doc_id in self.documents:
            lexical_scores[doc_id] = self.tfidf_score_normalized(query_terms, doc_id)

        # Normalize to 0-1
        max_lexical = max(lexical_scores.values()) if lexical_scores else 1
        if max_lexical > 0:
            lexical_scores = {k: v/max_lexical for k, v in lexical_scores.items()}

        # 2. Semantic scores (Embeddings)
        semantic_scores = self.semantic_score(query)

        # 3. ✨ Hybrid: 95% Semantic + 5% Lexical (הרצאה 6.2)
        final_scores = {}
        all_docs = set(lexical_scores.keys()) | set(semantic_scores.keys())

        for doc_id in all_docs:
            lexical = lexical_scores.get(doc_id, 0)
            semantic = semantic_scores.get(doc_id, 0)

            # Weighted combination (הרצאה 6.2)
            final_scores[doc_id] = alpha * semantic + (1 - alpha) * lexical

        # 4. Rank results
        ranked = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]

        # 5. ✨ De-duplicate chunks (אם יש chunking)
        # אם יש כמה chunks מאותו מסמך, קח רק את הטוב ביותר
        if self.use_chunking:
            seen_original_docs = set()
            deduplicated = []

            for doc_id, score in ranked:
                # בדוק אם זה chunk
                original_doc = doc_id.rsplit('_chunk_', 1)[0] if '_chunk_' in doc_id else doc_id

                if original_doc not in seen_original_docs:
                    seen_original_docs.add(original_doc)
                    deduplicated.append((doc_id, score))

            ranked = deduplicated[:top_k]

        return [{
            'doc_id': did,
            'score': float(score),
            'lexical_score': float(lexical_scores.get(did, 0)),
            'semantic_score': float(semantic_scores.get(did, 0)),
            'text': self.documents[did][:500],
            'metadata': self.doc_metadata[did]
        } for did, score in ranked]

    def search(self, query, top_k=5):
        """Default: 95% semantic, 5% lexical"""
        return self.hybrid_search(query, top_k, alpha=0.95)

print('✅ Improved indexer ready (PERFECT)')
print('   ✨ All features:')
print('      • Lemmatization (הרצאה 6 - slide 5)')
print('      • TF-IDF + Length Normalization (הרצאה 5 - slide 23)')
print('      • Document Chunking: 500 words (הרצאה 6.2 - slide 15)')
print('      • Hybrid Search: 95% Semantic + 5% Lexical (הרצאה 6.2)')
print('      • Selective Query Expansion (הרצאה 6)')

# ============================================================================
# RAG SYSTEM
# ============================================================================

class HybridRAGSystem:
    """✨ PERFECT RAG system with all improvements"""

    def __init__(self):
        self.use_chroma = CHROMADB_AVAILABLE

        if not self.use_chroma:
            # ✨ Use ImprovedIndexer with chunking!
            self.indexer = ImprovedIndexer(
                use_chunking=True,  # ✨ Enable chunking (הרצאה 6.2)
                chunk_size=500,     # 500 words per chunk
                overlap=50          # 50 words overlap
            )

        self.doc_count = 0

    def add_document(self, doc_id, text, metadata=None):
        """Add document (with automatic chunking)"""
        self.indexer.add_document(doc_id, text, metadata)
        self.doc_count += 1

        # בדוק אם המסמך חוטך
        if len(text.split()) > 500:
            num_chunks = len([d for d in self.indexer.documents if d.startswith(doc_id)])
            print(f"✓ Indexed: {metadata.get('title', doc_id)[:50]} [{num_chunks} chunks]")
        else:
            print(f"✓ Indexed: {metadata.get('title', doc_id)[:60]}")

    def search(self, query, top_k=3):
        """Search with hybrid (95% semantic)"""
        return self.indexer.hybrid_search(query, top_k, alpha=0.95)

    def query(self, question, top_k=3):
        """Query RAG"""
        results = self.search(question, top_k)

        if not results:
            return {'answer': 'לא נמצאו מאמרים רלוונטיים.', 'sources': [], 'papers_found': 0}

        search_summary = "**📊 מאמרים שנמצאו:**\n\n"
        for i, r in enumerate(results, 1):
            score = r.get('score', 0)
            title = r['metadata'].get('title', 'Unknown')

            # אם זה chunk, הראה מספר chunk
            if r['metadata'].get('is_chunk'):
                chunk_info = f" (Chunk {r['metadata']['chunk_num']+1}/{r['metadata']['total_chunks']})"
            else:
                chunk_info = ""

            search_summary += f"{i}. **{title}**{chunk_info} (Score: {score:.3f})\n"

            if 'lexical_score' in r and 'semantic_score' in r:
                search_summary += f"   Lexical: {r['lexical_score']:.3f} | Semantic: {r['semantic_score']:.3f}\n"

            if r['metadata'].get('doi'):
                search_summary += f"   DOI: {r['metadata']['doi']}\n"
            search_summary += "\n"

        try:
            context = '\n\n'.join([f"Paper {i+1}: {r['metadata'].get('title', 'Unknown')}\n{r['text']}" for i, r in enumerate(results)])
            prompt = f"""Based on these research papers, answer the question in Hebrew.

Question: {question}

Papers:
{context}

Provide a comprehensive answer in Hebrew."""

            response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[{'role': 'user', 'content': prompt}],
                max_tokens=800,
                temperature=0.3
            )
            answer = response.choices[0].message.content
            full_answer = search_summary + "\n---\n\n**💡 תשובה:**\n\n" + answer
        except Exception as e:
            full_answer = search_summary + f"\n⚠️ API Error: {str(e)}"

        return {'answer': full_answer, 'sources': [{'title': r['metadata'].get('title'), 'doi': r['metadata'].get('doi')} for r in results], 'papers_found': len(results)}

# Initialize
rag_system = HybridRAGSystem()
print('\n✅ RAG System ready (✨ PERFECT - שני שיפורים!)')
print('   Method: ImprovedIndexer')
print('   Model: all-MiniLM-L6-v2')
print('   Features:')
print('      • ✨ Document Chunking (500 words, 50 overlap) - הרצאה 6.2')
print('      • ✨ Optimized Alpha (95% Semantic) - הרצאה 6.2')
print('      • Lemmatization - הרצאה 6, slide 5')
print('      • TF-IDF + Normalization - הרצאה 5, slide 23')
print('      • Hybrid Search - הרצאה 6.2, slide 15')
print('      • Query Expansion - הרצאה 6')

🔄 Using ImprovedIndexer (ChromaDB disabled)
   ChromaDB installed but DISABLED
✅ Sentence-Transformers available
✅ Query expansion ready (selective)
✅ Document chunker ready
   ✨ Chunk size: 500 words, Overlap: 50 words (הרצאה 6.2)
✅ Improved indexer ready (PERFECT)
   ✨ All features:
      • Lemmatization (הרצאה 6 - slide 5)
      • TF-IDF + Length Normalization (הרצאה 5 - slide 23)
      • Document Chunking: 500 words (הרצאה 6.2 - slide 15)
      • Hybrid Search: 95% Semantic + 5% Lexical (הרצאה 6.2)
      • Selective Query Expansion (הרצאה 6)
   Loading SentenceTransformer (MiniLM)...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

   ✓ Model loaded

✅ RAG System ready (✨ PERFECT - שני שיפורים!)
   Method: ImprovedIndexer
   Model: all-MiniLM-L6-v2
   Features:
      • ✨ Document Chunking (500 words, 50 overlap) - הרצאה 6.2
      • ✨ Optimized Alpha (95% Semantic) - הרצאה 6.2
      • Lemmatization - הרצאה 6, slide 5
      • TF-IDF + Normalization - הרצאה 5, slide 23
      • Hybrid Search - הרצאה 6.2, slide 15
      • Query Expansion - הרצאה 6


## 📄 Cell 10: Document Loading Helpers

In [13]:
# @title
def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            return ''.join([page.extract_text() or '' for page in reader.pages])
    except Exception as e:
        print(f'⚠️ PDF error: {e}')
        return ''

def fetch_wikipedia_content(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        title = soup.find('h1', {'id': 'firstHeading'})
        content_div = soup.find('div', {'id': 'mw-content-text'})
        if not content_div:
            return None
        for el in content_div.find_all(['script', 'style', 'sup', 'table']):
            el.decompose()
        paras = content_div.find_all('p')
        text = (title.get_text() if title else '') + '\n\n'
        text += '\n\n'.join([re.sub(r'\[\d+\]', '', p.get_text()).strip() for p in paras if len(p.get_text()) > 50])
        return text
    except Exception as e:
        print(f'❌ Wikipedia error: {e}')
        return None

print('✅ Document helpers loaded')

✅ Document helpers loaded


## 📥 Cell 11: Load 5 PDFs + 2 Wikipedia Articles

In [14]:
# @title
DOCUMENTS = [
    {'type': 'pdf', 'file_id': '1evfiJEWg58DEy4N9HNrbPHRnSuOuEwPT', 'filename': 'paper1.pdf', 'doc_id': 'paddy', 'title': 'Classification of Paddy Plant Leaf Diseases', 'doi': '10.1016/j.procs.2025.04.393'},
    {'type': 'pdf', 'file_id': '1b5wjSmN_t6fYFkuI3lIFm_Rhmb4DYOFP', 'filename': 'paper2.pdf', 'doc_id': 'segmentation', 'title': 'Detection Using Image Segmentation', 'doi': '10.1016/j.inpa.2016.10.005'},
    {'type': 'pdf', 'file_id': '1uJ-kgrrYhA9eki73ckaI99I2g_YfIhjN', 'filename': 'paper3.pdf', 'doc_id': 'fig', 'title': 'Fig Plant Diseases Using CNN', 'doi': '10.32604/cmc.2025.063303'},
    {'type': 'pdf', 'file_id': '1bsrmA4A5ShkoEF98jQOiT0QuXf05ZqfZ', 'filename': 'paper4.pdf', 'doc_id': 'realtime', 'title': 'Real Time Monitoring System', 'doi': '10.1016/j.cropd.2024.100092'},
    {'type': 'pdf', 'file_id': '1VIHmV6p1judBH4Po-7qsSVch5vEtfS6O', 'filename': 'paper5.pdf', 'doc_id': 'encoder', 'title': 'Segmentation Encoder Techniques', 'doi': '10.2174/0118743315321139240627092707'},
    {'type': 'wikipedia', 'url': 'https://en.wikipedia.org/wiki/Plant_pathology', 'doc_id': 'wiki1', 'title': 'Plant Pathology (Wikipedia)'},
    {'type': 'wikipedia', 'url': 'https://en.wikipedia.org/wiki/Phytopathology', 'doc_id': 'wiki2', 'title': 'Phytopathology (Wikipedia)'}
]

print(f'📚 Loading {len(DOCUMENTS)} documents...\n')
for doc in DOCUMENTS:
    if doc['type'] == 'pdf':
        if not os.path.exists(doc['filename']):
            print(f"📥 Downloading {doc['title'][:40]}...")
            gdown.download(f"https://drive.google.com/uc?id={doc['file_id']}", doc['filename'], quiet=True)
        text = extract_text_from_pdf(doc['filename'])
        if text:
            rag_system.add_document(doc['doc_id'], text, {'title': doc['title'], 'doi': doc['doi'], 'source': 'PDF'})
    elif doc['type'] == 'wikipedia':
        text = fetch_wikipedia_content(doc['url'])
        if text:
            rag_system.add_document(doc['doc_id'], text, {'title': doc['title'], 'doi': 'N/A', 'source': 'Wikipedia'})

print(f'\n✅ Loaded {rag_system.doc_count} documents!')

📚 Loading 7 documents...

📥 Downloading Classification of Paddy Plant Leaf Disea...
✓ Indexed: Classification of Paddy Plant Leaf Diseases [9 chunks]
📥 Downloading Detection Using Image Segmentation...
✓ Indexed: Detection Using Image Segmentation [16 chunks]
📥 Downloading Fig Plant Diseases Using CNN...
✓ Indexed: Fig Plant Diseases Using CNN [25 chunks]
📥 Downloading Real Time Monitoring System...
✓ Indexed: Real Time Monitoring System [12 chunks]
📥 Downloading Segmentation Encoder Techniques...
✓ Indexed: Segmentation Encoder Techniques [9 chunks]
✓ Indexed: Plant Pathology (Wikipedia) [2 chunks]
✓ Indexed: Phytopathology (Wikipedia) [2 chunks]

✅ Loaded 7 documents!


## 🎨 Cell 12: Gradio Interface - 4 Screens

In [ ]:
# @title
# Cell 12: Gradio Interface Functions (+ Realtime Dashboard)

# ============================================================================
# REALTIME DASHBOARD FUNCTIONS (New!)
# ============================================================================

# Colors for realtime dashboard
COLOR_TEMP = "#1f77b4"   # blue
COLOR_HUM  = "#ff7f0e"   # orange
COLOR_SOIL = "#2ca02c"   # green

def load_iot_data_simple(feed: str, limit: int):
    """Load IoT data from server for realtime dashboard."""
    try:
        resp = requests.get(
            f"{BASE_URL}/history",
            params={"feed": feed, "limit": limit},
            timeout=30
        )
        data = resp.json()
        if "data" not in data or not data["data"]:
            return None

        df = pd.DataFrame(data["data"])
        if "created_at" not in df.columns or "value" not in df.columns:
            return None

        df["created_at"] = pd.to_datetime(df["created_at"], errors="coerce", utc=True)
        df["value"] = pd.to_numeric(df["value"], errors="coerce")
        df = df.dropna(subset=["created_at", "value"]).sort_values("created_at")

        return None if df.empty else df
    except:
        return None

def normalize_series(series):
    """Normalize series to 0-1 range."""
    mn, mx = float(series.min()), float(series.max())
    if mx - mn == 0:
        return series * 0.0
    return (series - mn) / (mx - mn)

def plant_dashboard_realtime(limit: int):
    """Generate realtime plant dashboard with status and plots."""
    try:
        # Load data for all sensors
        dfs = {
            "temperature": load_iot_data_simple("temperature", limit),
            "humidity": load_iot_data_simple("humidity", limit),
            "soil": load_iot_data_simple("soil", limit),
        }

        # Check for missing data
        missing = [k for k, v in dfs.items() if v is None]
        if missing:
            return "⚠️ Partial Data", f"Missing sensors or empty history: {', '.join(missing)}", None, None, None, None

        # Get latest values
        temp = float(dfs["temperature"]["value"].iloc[-1])
        hum = float(dfs["humidity"]["value"].iloc[-1])
        soil = float(dfs["soil"]["value"].iloc[-1])

        # Check plant status
        issues, warnings = [], []
        
        checks = [
            ("Temperature", temp, 18, 32, 1),
            ("Air humidity", hum, 35, 75, 3),
            ("Soil moisture", soil, 20, 60, 3),
        ]

        for name, value, low, high, margin in checks:
            if not (low <= value <= high):
                issues.append(f"{name} out of range ({value:.1f})")
            elif value <= low + margin or value >= high - margin:
                warnings.append(f"{name} near limit ({value:.1f})")

        # Determine overall status
        if issues:
            status = "🔴 Plant Status: Not OK"
            details_main = " ; ".join(issues)
        elif warnings:
            status = "🟡 Plant Status: Warning"
            details_main = " ; ".join(warnings)
        else:
            status = "🟢 Plant Status: OK"
            details_main = "All sensors are within valid ranges"

        details = (
            f"{details_main}\n"
            f"Latest values:\n"
            f"temp={temp:.1f}°C\n"
            f"humidity={hum:.1f}%\n"
            f"soil={soil:.1f}%"
        )

        # Create plots
        df_t, df_h, df_s = dfs["temperature"], dfs["humidity"], dfs["soil"]

        # Temperature plot
        fig_t = plt.figure(figsize=(7, 3.2))
        plt.plot(df_t["created_at"], df_t["value"], marker="o", color=COLOR_TEMP)
        plt.title("Temperature History")
        plt.xlabel("Time")
        plt.ylabel("°C")
        plt.grid(True)

        # Humidity plot
        fig_h = plt.figure(figsize=(7, 3.2))
        plt.plot(df_h["created_at"], df_h["value"], marker="o", color=COLOR_HUM)
        plt.title("Air Humidity History")
        plt.xlabel("Time")
        plt.ylabel("%")
        plt.grid(True)

        # Soil plot
        fig_s = plt.figure(figsize=(7, 3.2))
        plt.plot(df_s["created_at"], df_s["value"], marker="o", color=COLOR_SOIL)
        plt.title("Soil Moisture History")
        plt.xlabel("Time")
        plt.ylabel("%")
        plt.grid(True)

        # Combined normalized plot
        fig_c = plt.figure(figsize=(10, 3.4))
        plt.plot(df_t["created_at"], normalize_series(df_t["value"]), marker="o", label="Temperature (norm)", color=COLOR_TEMP)
        plt.plot(df_h["created_at"], normalize_series(df_h["value"]), marker="o", label="Humidity (norm)", color=COLOR_HUM)
        plt.plot(df_s["created_at"], normalize_series(df_s["value"]), marker="o", label="Soil (norm)", color=COLOR_SOIL)
        plt.title("Combined Trend (Normalized)")
        plt.xlabel("Time")
        plt.ylabel("Normalized Value (0–1)")
        plt.grid(True)
        plt.legend()

        return status, details, fig_t, fig_h, fig_s, fig_c

    except Exception as e:
        return "❌ Error", f"Failed to fetch data from server: {str(e)}", None, None, None, None



# ============================================================================
# PLANT ANALYSIS FUNCTION (Image + Sensors)
# ============================================================================

def analyze_plant(image, temp, humidity, soil):
    """Analyze plant image with sensor data."""
    
    # Run model prediction
    preds = clf(image)
    top = preds[0]
    label = top["label"]
    score = top["score"]
    
    alerts = []
    advice = []
    
    # --- Conditions based on sensors / user input ---
    if soil < 25:
        alerts.append("⚠️ Low soil moisture")
        advice.append("💧 Recommendation: irrigate / water the plant")
    
    if humidity > 80:
        alerts.append("⚠️ High humidity")
        advice.append("🌬️ Recommendation: improve ventilation (reduce fungal risk)")
    
    if temp > 40:
        alerts.append("⚠️ High temperature")
        advice.append("☀️ Recommendation: move the plant to a shaded area")
    
    # --- Color status ("flag") based on image prediction ---
    # Simple rule: if label contains "healthy" => good (green), else bad (red)
    is_bad = ("healthy" not in label.lower())
    
    status_html = (
        "<div style='padding: 20px; border-radius: 10px; text-align: center; "
        f"background: {'#fee2e2' if is_bad else '#d1fae5'}; "
        f"color: {'#991b1b' if is_bad else '#065f46'}; "
        "font-size: 24px; font-weight: bold; margin: 10px 0;'>"
        f"{'🔴 Plant status: BAD' if is_bad else '🟢 Plant status: GOOD'}"
        "</div>"
    )
    
    if not alerts:
        alerts.append("✅ Status looks normal")
    
    return (
        f"🔍 Detected disease: {label} ({score:.2%})",
        status_html,
        "\n".join(alerts),
        "\n".join(advice) if advice else "No specific recommendations at this time."
    )

# ============================================================================
# DASHBOARD FUNCTIONS (keep original)
# ============================================================================

def dashboard_screen():
    """Load all data and return comprehensive dashboard."""
    df = load_data_from_firebase()

    if df.empty:
        empty_msg = "<div style='padding: 20px; text-align: center;'>אין נתונים. לחץ על Sync Data!</div>"
        return empty_msg, None, None, None, None, None, None, None, None, None, None, None, None

    # Generate all visualizations
    kpi = create_kpi_cards(df)
    stats = create_stat_cards_html(df)
    ts = create_time_series_plot(df)
    corr_card, corr_plot = calculate_correlations(df)
    hourly_card, hourly_plot = hourly_patterns(df)
    daily_card, daily_plot = daily_patterns(df)
    dist_card, dist_plot = distribution_analysis(df)
    scatter_card, scatter_plot = scatter_analysis(df)

    return kpi, stats, ts, corr_card, corr_plot, hourly_card, hourly_plot, daily_card, daily_plot, dist_card, dist_plot, scatter_card, scatter_plot

def dashboard_moving_avg(variable):
    """Generate moving average plot for selected variable."""
    df = load_data_from_firebase()
    if df.empty:
        return None, "אין נתונים"
    ma_card, ma_plot = time_series_decomposition(df, variable)
    return ma_card, ma_plot

def generate_report_screen():
    """Generate and download DOCX report."""
    df = load_data_from_firebase()
    if df.empty:
        return "אין נתונים ליצירת דוח", None

    try:
        output_path = report_gen.create_docx_report(df)
        return f"✅ דוח נוצר בהצלחה!\n📥 הורד את הקובץ:", output_path
    except Exception as e:
        return f"❌ שגיאה: {str(e)}", None

# ============================================================================
# SEARCH ENGINE (Simple - No AI)
# ============================================================================

def simple_search_screen(query, num_papers=7):
    """Simple search engine - returns formatted results."""
    if not query:
        return 'הזן שאילתא לחיפוש במאמרים'

    # Search
    num_papers = int(num_papers) if num_papers else 7
    results = rag_system.search(query, top_k=num_papers)

    if not results:
        return "**⚠️ לא נמצאו מאמרים רלוונטיים**"

    # Format output
    output = f"**🔍 נמצאו {len(results)} מאמרים:**\n\n"
    output += "---\n\n"

    for i, r in enumerate(results, 1):
        score = r.get('score', 0)
        title = r['metadata'].get('title', 'Unknown')
        doi = r['metadata'].get('doi', 'N/A')
        text_preview = r['text'][:200] + "..." if len(r['text']) > 200 else r['text']

        output += f"### 📄 Result #{i}\n\n"
        output += f"**Title:** {title}\n\n"
        if isinstance(score, float):
            output += f"**Score:** {score:.3f}\n\n"
        output += f"**DOI:** {doi}\n\n"
        output += f"**Preview:** {text_preview}\n\n"
        output += "---\n\n"

    return output

# ============================================================================
# RAG CHAT (With History & IoT Integration)
# ============================================================================

# Chat history storage
chat_history = []

def rag_chat_respond(message, history):
    """RAG chat with sensor data integration."""
    if not message.strip():
        return history

    try:
        # Search papers
        results = rag_system.search(message, top_k=3)

        # Get sensor data
        df = load_data_from_firebase()
        sensor_context = ""
        if not df.empty:
            latest = df.iloc[-1]
            sensor_context = f"""\n\nCurrent sensor readings:
- Temperature: {latest['temperature']:.1f}°C
- Humidity: {latest['humidity']:.1f}%
- Soil Moisture: {latest['soil']:.1f}%"""

        # Build context
        papers_context = "\n\n".join([
            f"Paper {i+1} ({r['metadata'].get('title')}):\n{r['text'][:400]}"
            for i, r in enumerate(results[:3])
        ]) if results else "No relevant papers found."

        # Create prompt
        prompt = f"""You are an agricultural AI assistant.

Research papers:
{papers_context}
{sensor_context}

Current question: {message}

Instructions:
1. Answer in Hebrew
2. Reference sensor data if relevant to the question
3. Consider previous conversation context
4. Cite papers when using their information
5. Be concise and helpful

Answer:"""

        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[{'role': 'user', 'content': prompt}],
            max_tokens=800,
            temperature=0.3
        )
        bot_response = response.choices[0].message.content

        # Add sources footnote
        sources_list = "\n\n---\n**📚 מקורות:**\n"
        for i, r in enumerate(results[:3], 1):
            sources_list += f"{i}. {r['metadata'].get('title')} (Score: {r.get('score', 0):.3f})\n"

        bot_response += sources_list

    except Exception as e:
        print(f"⚠️ Error: {e}")
        bot_response = f"מצטער, הייתה שגיאה: {str(e)}"

    # Add to history
    history.append((message, bot_response))
    return history

def clear_chat():
    """Clear chat history."""
    global chat_history
    chat_history = []
    return []

# ============================================================================
# SYNC DATA
# ============================================================================

def sync_screen():
    msg, count = sync_new_data_from_server()
    return msg

print('✅ All screen functions loaded!')
print('   📊 Dashboard')
print('   📄 Report Generator')
print('   🖼️ Image Detection (Simple with Sensors)')
print('   🔍 Search Engine')
print('   💬 RAG Chat')
print('   🔄 Data Sync')

## 🚀 Cell 13: Launch Gradio App

In [ ]:
# @title
# Cell 13: Launch Gradio App (with Realtime Dashboard as first tab)

with gr.Blocks(css=CUSTOM_CSS, title='🌱 Plant System') as app:
    gr.Markdown('# 🌱 Smart Plant Disease Detection System')
    gr.Markdown('**מערכת חכמה לזיהוי מחלות צמחים עם IoT, AI ו-RAG**')

    with gr.Tabs():
        # ====================================================================
        # ⭐ Tab 0: REALTIME DASHBOARD (NEW - FIRST TAB!)
        # ====================================================================
        with gr.Tab('🌿 Realtime Dashboard'):
            gr.Markdown('### 🌱 CloudGarden – Plant Monitoring Dashboard')
            gr.Markdown('**Real-time plant status with automatic health checks**')
            
            samples = gr.Slider(
                minimum=1,
                maximum=200,
                value=20,
                step=1,
                label="Number of Samples (used for all graphs)"
            )
            
            update_btn = gr.Button(
                '🔄 Update Plant Dashboard',
                variant='primary',
                size='lg'
            )
            
            # Status outputs
            gr.Markdown('### 📊 Plant Status')
            overall_status = gr.Textbox(
                label="Overall Status",
                lines=1,
                placeholder="Click 'Update Plant Dashboard' to evaluate plant status"
            )
            overall_info = gr.Textbox(
                label="Status Details",
                lines=4,
                placeholder="Detailed plant analysis will appear here"
            )
            
            # Info cards
            with gr.Row():
                gr.Markdown("""
#### 🌿 Plant Status Legend
- **🟢 Healthy** – All sensor values within normal ranges
- **🟡 Warning** – At least one value near threshold
- **🔴 Not OK** – One or more values out of range

Status is calculated automatically from sensor data
                """)
                
                gr.Markdown("""
#### ℹ️ Valid Value Ranges
- **🌡️ Temperature:** 18–32°C
- **💧 Air Humidity:** 35–75%
- **🌱 Soil Moisture:** 20–60%

⚠️ Values outside these ranges are considered abnormal
                """)
            
            # Plots
            gr.Markdown('### 📈 Sensor Graphs')
            
            with gr.Row():
                plot_temp = gr.Plot(label="Temperature History")
                plot_hum = gr.Plot(label="Air Humidity History")
            
            with gr.Row():
                plot_soil = gr.Plot(label="Soil Moisture History")
                plot_combined = gr.Plot(label="Combined Trend (Normalized)")
            
            # Wire up button
            update_btn.click(
                fn=plant_dashboard_realtime,
                inputs=[samples],
                outputs=[overall_status, overall_info, plot_temp, plot_hum, plot_soil, plot_combined]
            )

        # ====================================================================
        # Tab 1: IoT Dashboard (was first, now second)
        # ====================================================================
        with gr.Tab('📊 IoT Dashboard'):
            gr.Markdown('### 📈 Comprehensive Sensor Analytics')

            refresh_btn = gr.Button('🔄 Refresh All Data', variant='primary', size='lg')

            # KPI Cards
            gr.Markdown('#### 📌 Current Readings')
            kpi_html = gr.HTML()

            # Statistics Cards
            gr.Markdown('#### 📊 Statistical Summary')
            stats_html = gr.HTML()

            # Main Time Series
            gr.Markdown('#### 📈 Time Series Overview')
            ts_plot = gr.Plot()

            # Correlation Analysis
            gr.Markdown('#### 🔗 Correlation Analysis')
            corr_card = gr.HTML()
            corr_plot = gr.Plot()

            # Hourly Patterns
            gr.Markdown('#### ⏰ Hourly Patterns')
            hourly_card = gr.HTML()
            hourly_plot = gr.Plot()

            # Daily Trends
            gr.Markdown('#### 📅 Daily Trends')
            daily_card = gr.HTML()
            daily_plot = gr.Plot()

            # Distribution Analysis
            gr.Markdown('#### 📊 Distribution Analysis (Histograms)')
            dist_card = gr.HTML()
            dist_plot = gr.Plot()

            # Scatter Analysis
            gr.Markdown('#### 🔵 Scatter Analysis')
            scatter_card = gr.HTML()
            scatter_plot = gr.Plot()

            # Moving Averages
            gr.Markdown('#### 📉 Moving Averages')
            with gr.Row():
                ma_variable = gr.Dropdown(
                    choices=['temperature', 'humidity', 'soil'],
                    value='temperature',
                    label='Select Variable'
                )
                ma_btn = gr.Button('Generate Moving Average')
            ma_card = gr.HTML()
            ma_plot = gr.Plot()

            # Wire up refresh button
            refresh_btn.click(
                dashboard_screen,
                outputs=[
                    kpi_html, stats_html, ts_plot,
                    corr_card, corr_plot,
                    hourly_card, hourly_plot,
                    daily_card, daily_plot,
                    dist_card, dist_plot,
                    scatter_card, scatter_plot
                ]
            )

            # Wire up moving average
            ma_btn.click(
                dashboard_moving_avg,
                inputs=ma_variable,
                outputs=[ma_card, ma_plot]
            )

        # ====================================================================
        # Tab 2: Report Generator
        # ====================================================================
        with gr.Tab('📄 Generate Report'):
            gr.Markdown('### יצירת דוח Word אוטומטי')
            gr.Markdown('הדוח כולל: סטטיסטיקות, ניתוח AI, והמלצות')
            report_btn = gr.Button('📥 Generate & Download Report', variant='primary')
            report_status = gr.Textbox(label='Status', lines=3)
            report_file = gr.File(label='📥 Download Report')
            report_btn.click(generate_report_screen, outputs=[report_status, report_file])

        # ====================================================================
        # Tab 3: Plant Disease Detection
        # ====================================================================
        with gr.Tab('🖼️ Plant Disease Detection'):
            gr.Markdown('### 🌿 Plant Disease Detection from Image')
            gr.Markdown('**Upload a plant image and provide sensor readings for comprehensive analysis**')
            
            with gr.Row():
                # Left column: Image
                with gr.Column(scale=1):
                    image_input = gr.Image(
                        type="filepath",
                        label="📸 Upload plant image",
                        show_label=True,
                        sources=["upload"],
                        height=300
                    )
                
                # Right column: Sensor inputs
                with gr.Column(scale=1):
                    gr.Markdown('#### 🌡️ Environmental Conditions')
                    temp_input = gr.Slider(
                        minimum=0,
                        maximum=45,
                        value=25,
                        label="Temperature (°C)",
                        info="Current air temperature"
                    )
                    humidity_input = gr.Slider(
                        minimum=0,
                        maximum=100,
                        value=50,
                        label="Humidity (%)",
                        info="Relative air humidity"
                    )
                    soil_input = gr.Slider(
                        minimum=0,
                        maximum=100,
                        value=50,
                        label="Soil Moisture (%)",
                        info="Soil water content"
                    )
            
            analyze_btn = gr.Button(
                '🔍 Analyze Plant',
                variant='primary',
                size='lg'
            )
            
            # Outputs
            gr.Markdown('### 📊 Analysis Results')
            
            with gr.Row():
                diagnosis_output = gr.Textbox(
                    label="🔬 Diagnosis",
                    placeholder="Plant disease diagnosis will appear here",
                    lines=2
                )
            
            status_output = gr.HTML(label="Status")
            
            with gr.Row():
                alerts_output = gr.Textbox(
                    label="⚠️ Alerts",
                    lines=5,
                    placeholder="Sensor alerts and warnings will appear here"
                )
                recommendations_output = gr.Textbox(
                    label="💡 Recommendations",
                    lines=5,
                    placeholder="Care and treatment recommendations will appear here"
                )
            
            # Wire up analyze button
            analyze_btn.click(
                analyze_plant,
                inputs=[image_input, temp_input, humidity_input, soil_input],
                outputs=[diagnosis_output, status_output, alerts_output, recommendations_output]
            )

        # ====================================================================
        # Tab 4: Search Engine
        # ====================================================================
        with gr.Tab('🔍 Search Engine'):
            gr.Markdown('### מנוע חיפוש במאמרים מדעיים')
            gr.Markdown('**חיפוש פשוט - רק תוצאות, ללא AI**')

            with gr.Row():
                search_query = gr.Textbox(
                    label='שאילתא',
                    placeholder='לדוגמה: CNN, tomato diseases, plant detection',
                    lines=2,
                    scale=4
                )
                search_papers_count = gr.Slider(
                    minimum=1,
                    maximum=7,
                    value=5,
                    step=1,
                    label='📚 מספר מאמרים',
                    scale=1
                )

            search_btn = gr.Button('🔍 Search', variant='primary', size='lg')
            search_output = gr.Markdown(label='תוצאות חיפוש')

            search_btn.click(
                simple_search_screen,
                inputs=[search_query, search_papers_count],
                outputs=search_output
            )
            search_query.submit(
                simple_search_screen,
                inputs=[search_query, search_papers_count],
                outputs=search_output
            )

        # ====================================================================
        # Tab 5: RAG Chat
        # ====================================================================
        with gr.Tab('💬 RAG Chat'):
            gr.Markdown('### צ\'אט חכם עם AI')
            gr.Markdown('**שאל שאלות - המערכת משתמשת בנתוני החיישנים + מאמרים**')

            chatbot = gr.Chatbot(
                label="💬 Conversation",
                height=500,
                show_label=True,
                bubble_full_width=False
            )

            with gr.Row():
                chat_input = gr.Textbox(
                    label="Your Message",
                    placeholder="שאל שאלה על הצמח, המחלות, או נתוני החיישנים...",
                    lines=2,
                    scale=4
                )
                chat_send = gr.Button("📤 Send", variant="primary", scale=1)

            with gr.Row():
                chat_clear = gr.Button("🗑️ Clear Chat", size="sm")

            # Wire up chat
            chat_send.click(
                rag_chat_respond,
                inputs=[chat_input, chatbot],
                outputs=chatbot
            ).then(
                lambda: "",
                outputs=chat_input
            )

            chat_input.submit(
                rag_chat_respond,
                inputs=[chat_input, chatbot],
                outputs=chatbot
            ).then(
                lambda: "",
                outputs=chat_input
            )

            chat_clear.click(clear_chat, outputs=chatbot)

        # ====================================================================
        # Tab 6: Sync Data
        # ====================================================================
        with gr.Tab('🔄 Sync Data'):
            gr.Markdown('### סנכרון נתונים מהשרת')
            gr.Markdown('לחץ לסנכרון נתוני IoT חדשים')

            sync_btn = gr.Button('🔄 Sync New Data', variant='primary', size='lg')
            sync_output = gr.Textbox(label='Status', lines=5)

            sync_btn.click(sync_screen, outputs=sync_output)

    # Footer
    gr.Markdown("""
---
**🌟 Features:**
- 🌿 **NEW: Realtime Dashboard** - Live plant status with auto health checks
- 📊 Real-time IoT monitoring (Firebase)
- 🖼️ AI Image Detection with Environmental Sensors
- 📚 Research Paper Search (RAG)
- 💬 Intelligent Chat (RAG + Cerebras)
- 📄 Automated Reports (DOCX)

**✨ Realtime Dashboard:**
- Automatic plant health evaluation
- Color-coded status (🔴 BAD / 🟡 WARNING / 🟢 OK)
- 4 real-time graphs (Temperature, Humidity, Soil, Combined)
- Smart thresholds with margin checking
    """)

# Launch
app.launch(share=True, debug=True)
